In [1]:
import sys
sys.path.append("../")
%matplotlib tk

import numpy as np
from scipy.constants import c, e, m_p
import matplotlib.pyplot as plt


import PyCERNmachines.CERNmachines as m
from PySUSSIX import Sussix, ParallelSussix

from PyHEADTAIL.spacecharge.transverse_spacecharge import TransverseSpaceCharge
from PyHEADTAIL.field_maps.Transverse_Efield_map import Transverse_Efield_map

from PyPIC.FFT_OpenBoundary_SquareGrid import FFT_OpenBoundary_SquareGrid
from PyHEADTAIL.particles.slicing import UniformBinSlicer

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from PyHEADTAIL.particles.generators import ParticleGenerator, gaussian2D_asymmetrical



PyHEADTAIL v1.4.1-19-g198c5df2bd




### tune analysis

In [2]:
def tune_analysis(bunch, machine, x_i, xp_i, y_i, yp_i):
    n_turns = x_i.shape[1]

    qx_i = np.empty(bunch.macroparticlenumber)
    qy_i = np.empty(bunch.macroparticlenumber)

    spectrum_x = np.abs(np.fft.fft(x_i, axis = 1))[:,:n_turns/2]
    tune_peak_x = np.float_(np.argmax(spectrum_x, axis=1))/float(n_turns)

    spectrum_y = np.abs(np.fft.fft(y_i, axis = 1))[:,:n_turns/2]
    tune_peak_y = np.float_(np.argmax(spectrum_y, axis=1))/float(n_turns)

    print 'analysing particle spectra ... this may take some time.'
    for p_idx in xrange(bunch.macroparticlenumber):

        SX = Sussix()
        #print tune_peak_x[p_idx]
        SX.sussix_inp(nt1=1, nt2=n_turns, idam=2, ir=0, tunex=tune_peak_x[p_idx], tuney=tune_peak_y[p_idx])

        SX.sussix(x_i[p_idx,:], xp_i[p_idx,:],
                  y_i[p_idx,:], yp_i[p_idx,:],
                  x_i[p_idx,:], xp_i[p_idx,:])
        qx_i[p_idx] = SX.ox[0]
        qy_i[p_idx] = SX.oy[0]

        sys.stdout.write('\rparticle %d'%p_idx)

    x_centroid = np.mean(x_i, axis=0)
    xp_centroid = np.mean(xp_i, axis=0)
    y_centroid = np.mean(y_i, axis=0)
    yp_centroid = np.mean(yp_i, axis=0)

    #print x_centroid.shape

    SX = Sussix()
    SX.sussix_inp(nt1=1, nt2=n_turns, idam=2, ir=0, tunex=machine.Q_x%1, tuney=machine.Q_y%1)
    SX.sussix(x_centroid, xp_centroid,
              y_centroid, yp_centroid,
              x_centroid, xp_centroid) 
    qx_centroid = SX.ox[0]
    qy_centroid = SX.oy[0]

    return qx_i, qy_i, qx_centroid, qy_centroid


def tune_analysis_new(bunch, machine, x_i, xp_i, y_i, yp_i):
    n_turns = x_i.shape[1]

    qx_i = np.empty(bunch.macroparticlenumber)
    qy_i = np.empty(bunch.macroparticlenumber)

    spectrum_x = np.abs(np.fft.fft(x_i, axis = 1))[:,:n_turns/2]
    tune_peak_x = np.float_(np.argmax(spectrum_x, axis=1))/float(n_turns)

    spectrum_y = np.abs(np.fft.fft(y_i, axis = 1))[:,:n_turns/2]
    tune_peak_y = np.float_(np.argmax(spectrum_y, axis=1))/float(n_turns)

    print 'analysing particle spectra ... this may take some time.'
    for p_idx in xrange(bunch.macroparticlenumber):

        SX = ParallelSussix()
        SX.sussix(x_i[p_idx,:], xp_i[p_idx,:],
                  y_i[p_idx,:], yp_i[p_idx,:],
                  x_i[p_idx,:], xp_i[p_idx,:],
                  nt1=1, nt2=n_turns, idam=2, ir=0, tunex=tune_peak_x[p_idx], tuney=tune_peak_y[p_idx]) 
        qx_i[p_idx] = SX.ox[0]
        qy_i[p_idx] = SX.oy[0]

        sys.stdout.write('\rparticle %d'%p_idx)

    x_centroid = np.mean(x_i, axis=0)
    xp_centroid = np.mean(xp_i, axis=0)
    y_centroid = np.mean(y_i, axis=0)
    yp_centroid = np.mean(yp_i, axis=0)

    qx_centroid = SX.ox[0]
    qy_centroid = SX.oy[0]

    return qx_i, qy_i, qx_centroid, qy_centroid
    
    

In [3]:
def plot_tune_spread(machine, qx_i, qy_i, qx_centroid, qy_centroid):
    plt.plot(np.abs(qx_i), np.abs(qy_i), '.')
    plt.plot([np.modf(machine.Q_x)[0]], [np.modf(machine.Q_y)[0]], 'ro', label='Bare tune')
    #if flag_initial_displacement:
    #    plt.plot([np.abs(qx_centroid)], [np.abs(qy_centroid)], 'xg', markersize = 10, label='Coherent tune')
    #plt.plot([.15, .30], [.15, .30], 'k')
    plt.legend(loc='lower right')
    plt.xlabel('$Q_x$');plt.ylabel('$Q_y$')
    plt.axis('equal')
    plt.show()

### define the machine (PS) and create a matched gaussian bunch

In [4]:
macroparticlenumber = 200
macroparticlenumber_for_field_calculation = 1000000
charge    = e
mass      = m_p
n_turns = 128

machine = m.PS(Q_x=6.35, Q_y=6.27, gamma=2.49, n_segments=20, longitudinal_focusing='linear', charge=charge, mass=mass)

intensity = 160e10/2
epsn_x    = 2.5e-6
epsn_y    = 2.5e-6
sigma_z   = 180e-9 /4 * (machine.beta * c)

bunch = machine.generate_6D_Gaussian_bunch(n_macroparticles=macroparticlenumber, intensity=intensity,
                                          epsn_x=epsn_x, epsn_y=epsn_y, sigma_z=sigma_z)
bunch_for_field = machine.generate_6D_Gaussian_bunch(n_macroparticles=macroparticlenumber_for_field_calculation,
                                                     intensity=intensity,
                                          epsn_x=epsn_x, epsn_y=epsn_y, sigma_z=sigma_z)

# store the initial positions
x_init = bunch.x.copy()
xp_init = bunch.xp.copy()
y_init = bunch.y.copy()
yp_init = bunch.yp.copy()
z_init = bunch.z.copy()
dp_init = bunch.dp.copy()


Synchrotron init. From kwargs: charge = 1.602176565e-19
Synchrotron init. From kwargs: mass = 1.672621777e-27
Synchrotron init. From kwargs: Q_y = 6.27
Synchrotron init. From kwargs: n_segments = 20
Synchrotron init. From kwargs: gamma = 2.49
Synchrotron init. From kwargs: Q_x = 6.35


### create the maps:  insert space-charge element

In [5]:
# longitudinal slicing of the bunch
z_cut = np.max(np.abs(bunch_for_field.z))*1.2
n_slices = 50
slicer = UniformBinSlicer(n_slices = n_slices, z_cuts=(-z_cut, z_cut))

# spacecharge object
spacechargepic = FFT_OpenBoundary_SquareGrid(x_aper=bunch_for_field.sigma_x()*4, 
                                             y_aper=bunch_for_field.sigma_y()*4,
                                             Dh=(bunch_for_field.sigma_x()+bunch_for_field.sigma_y())/2/10)
spacecharge = TransverseSpaceCharge(L_interaction=1, slicer=slicer, pyPICsolver=spacechargepic)
spacecharge.save_distributions_last_track = True
spacecharge.save_potential_and_field = True

# track the bunch through once to save the fields
spacecharge.track(bunch_for_field)

# create an electric field map object to store the field
rescale_strength_by = 1.
L_interaction = machine.circumference/len(machine.transverse_map)
efieldmap = Transverse_Efield_map(xg = spacecharge.pyPICsolver.xg, yg = spacecharge.pyPICsolver.yg, 
                Ex=spacecharge.Ex_last_track, Ey=spacecharge.Ey_last_track, n_slices=n_slices, z_cut=z_cut, 
                L_interaction=L_interaction, flag_clean_slices = True, wrt_slice_centroid = False)

# install ecloud field kick after each segment of the machine
machine.install_after_each_transverse_segment(efieldmap)

Start PIC init.:
FFT, Open Boundary, Square Grid
PyPIC Version 1.03
Using PyFFTW
PyPIC Version 1.03


../PyPIC/PyPIC_Scatter_Gather.py:70: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if xg!=None and xg!=None:


### movie of charge density

In [6]:
slices = bunch_for_field.get_slices(spacecharge.slicer)
show_movie = False
if show_movie:
    import time
    n_frames = spacecharge.rho_last_track.shape[0]
    plt.close(1)
    plt.figure(1, figsize=(8,8))
    vmax = np.max(spacecharge.rho_last_track[:])
    vmin = np.min(spacecharge.rho_last_track[:])
    for ii in xrange(n_frames-1, 0, -1):
        plt.subplot2grid((10,1),(0,0), rowspan=3)
        plt.plot(slices.z_centers, slices.n_macroparticles_per_slice/np.max(slices.n_macroparticles_per_slice))
        plt.xlabel('z [m]');plt.ylabel('Long. profile')
        plt.axvline(x = slices.z_centers[ii], color='r')
        plt.subplot2grid((10,1),(4,0), rowspan=6)
        plt.pcolormesh(spacecharge.pyPICsolver.xg*1e3, spacecharge.pyPICsolver.yg*1e3, 
                       spacecharge.rho_last_track[ii,:,:].T, vmax=vmax, vmin=vmin)
        plt.xlabel('x [mm]');plt.ylabel('y [mm]')
        plt.axis('equal')
        plt.subplots_adjust(hspace=.5)
        plt.draw()
        #plt.show()
        time.sleep(.2)

### track the particles

In [7]:
# particle positions for tune analysis
x_i = np.empty((macroparticlenumber, n_turns))
xp_i = np.empty((macroparticlenumber, n_turns))
y_i = np.empty((macroparticlenumber, n_turns))
yp_i = np.empty((macroparticlenumber, n_turns))

bunch.update({'x': x_init, 'xp': xp_init, 'y': y_init, 'yp': yp_init})


print 'bunch: epsn_x=',bunch.epsn_x()
# track and store
for i in range(n_turns):    
    machine.track(bunch)
    
    sys.stdout.write('\rturn %d'%i)
    sys.stdout.flush()
    
    x_i[:,i] = bunch.x[:]
    xp_i[:,i] = bunch.xp[:]
    y_i[:,i] = bunch.y[:]
    yp_i[:,i] = bunch.yp[:]
print '\nDONE'
print 'bunch: epsn_x=',bunch.epsn_x()


bunch: epsn_x= 2.3858852185e-06
turn 127
DONE
bunch: epsn_x= 2.42284688097e-06


## plot the tune diagram

In [8]:
qx_i, qy_i, qx_centroid, qy_centroid = tune_analysis(bunch,  machine, x_i, xp_i, y_i, yp_i)
plot_tune_spread(machine, qx_i, qy_i, qx_centroid, qy_centroid)

analysing particle spectra ... this may take some time.
particle 199

## plot efield

In [12]:
slice_2_plot = np.floor(0.5*slicer.n_slices)
fig = plt.figure(6)
fig.clf()
ax = fig.gca(projection='3d')
X = efieldmap.pic.xg
Y = efieldmap.pic.yg
X, Y = np.meshgrid(X, Y)
surf = ax.plot_surface(X, Y, efieldmap.Ex[slice_2_plot,:,:].T, rstride=1, cstride=1, cmap=cm.jet,
        linewidth=0.2, antialiased=True)
# plt.plot(efieldmap.Ex[1]);
plt.xlabel('x [m]')
plt.ylabel('y [m]')
ax.set_zlabel('Ex [V/m]')

In [13]:
print qx_i[0:10]
print qy_i[0:10]

[ 0.29819002  0.28563708  0.29412592  0.34216207  0.30933696  0.30077264
  0.28611589  0.2565425   0.30959432  0.2348015 ]
[ 0.23215686  0.21632307  0.220087    0.26152907  0.22880899  0.21605076
  0.20120458  0.18572925  0.22130615  0.15561439]


In [14]:
import pickle
fileObject = open('bunch.pickle','wb') 

# this writes the object a to the
# file named 'testfile'
pickle.dump(bunch,fileObject)   

# here we close the fileObject
fileObject.close()


np.savetxt('x_i.txt', x_i)
np.savetxt('xp_i.txt', xp_i)
np.savetxt('y_i.txt', y_i)
np.savetxt('yp_i.txt', yp_i)